### 棋盘的实现——来自`Board.py`源代码
修改了一部分，进行适当剪枝提高算法效率

In [35]:
import numpy as np
from collections import deque


np.set_printoptions(threshold=np.inf)
class Board:
    # 初始化棋盘大小、颜色和棋盘数量
    def __init__(self, size=8, num=100, colors=np.array(['R', 'G', 'B', 'Y'])):
        '''
        初始化棋盘
        size: 棋盘大小，默认值为8
        num: 一个测试中生成的子棋盘数量，默认值为100
        colors: 可用颜色列表，默认为 ['R', 'G', 'B', 'Y']
        '''
        self.size = size
        self.colors = colors  # 可选颜色数组
        self.board = np.concatenate(
            [self.generate_board() for i in range(num)]).T  # 连接多个棋盘

    # 生成一个新棋盘，并保证该棋盘不存在连续三个相邻相同的颜色方块
    def generate_board(self):
        '''
        生成一个新的随机子棋盘，并且保证没有任何相邻元素的颜色相同。
        返回值：
        np.array：返回新生成的矩阵；
        '''
        a = np.concatenate([np.full((self.size // 2, self.size // 2), self.colors[i])
                            for i in range(4)]).reshape(self.size ** 2, 1)

        np.random.shuffle(a)  # 打乱颜色块的顺序
        new_array = a.reshape(self.size, self.size)

        while self.check(new_array):  # 检查是否存在连续三个相邻元素
            for i, j in self.check(new_array):
                # 随机替换掉其中一个同色元素
                new_array[i][j] = new_array[i - np.random.randint(
                    1, self.size-1)][j - np.random.randint(1, self.size-1)]

        return new_array

    def current_board(self):
        '''
        返回当前棋盘的状态副本。
        返回值：
        np.array：返回当前棋盘的状态副本矩阵；
        '''
        return self.board.copy()

    def copy(self, copy_size=None):
        """
        copy_size是拷贝的棋盘数目（太多影响性能）
        """

        if copy_size is None:
            other = Board(
                self.size, self.board.shape[1] // self.size, self.colors)
            other.board = self.board.copy()
        else:
            other = Board(self.size, copy_size, self.colors)
            other.board = self.board[:, :copy_size * self.size].copy()
        other.lazylines = self.lazylines
        return other

    # 获取主要棋盘（size*size）
    def main_board(self):
        '''
        返回当前棋盘的主要区域（正方形区域），长度为size的二维numpy数组。
        返回值：
        np.array: 返回当前棋盘状态下的主要区域。
        '''
        return self.board[:self.size, :self.size]

    # 检查棋盘是否存在连续三个相邻的同色元素
    def check(self, array):
        '''
        返回在电路中重复的位置。
        参数：
        array: 以二维数组形式输入的元素列表；
        返回值：
        set: (i, j)位置集合，即 (i, j) 是重复位置。
        '''
        repeats = set()
        for i in range(1, self.size - 1):  # 遍历行
            for j in np.arange(self.size):
                a = array[i - 1:i + 2, j]
                b = (a == array[i, j])
                if np.sum(b) == 3:
                    repeats.add((i, j))
        for i in range(1, self.size - 1):  # 遍历列
            for j in np.arange(self.size):
                a = array[j, i - 1:i + 2]
                b = (a == array[j, i])
                if np.sum(b) == 3:
                    repeats.add((j, i))
        return repeats

    def move(self, row1, col1, row2, col2):
        '''
        在棋盘上移动两个颜色方块，并存储每一步操作。
        参数：
        row1, col1: 第一块方块的行和列索引
        row2, col2: 第二块方块的行和列索引
        返回值：
        list: 包含下面 4 个元素的切片列表
            - 当前棋盘状态
            - 第一块方块的坐标和颜色信息
            - 第二块方块的坐标和颜色信息
            - 移动后的新棋盘状态
       '''
        self.board[row1, col1], self.board[row2, col2] = \
            self.board[row2, col2], self.board[row1, col1]   # 调换两个棋子
        # 返回当前操作的所有步骤，供后续检查和分析使用
        self.lazylines = set([row1, row2])

    def scan_for_connected(self):
        matrix = self.board
        visited = np.zeros_like(self.main_board(), dtype=np.uint8)
        res = []
        temp_lines = set()
        # 用广搜获得所有主盘面上联通的元素，以[[(x11,y11),...,(x1n,y1n)],...,[(xn1,yn1),...,(xnn,ynn)]]的形式储存所有联通的瞳色元素
        for i in self.lazylines:
            for j in range(self.size):
                if not visited[i, j] and matrix[i, j] != 'Q':
                    visited[i, j] = 1
                    temp = []
                    queue = deque([(i, j)])
                    while queue:
                        x, y = queue.popleft()
                        temp.append((x, y))
                        if x + 1 < self.size and not visited[x + 1, y] and matrix[x + 1, y] == matrix[x, y]:
                            queue.append((x + 1, y))
                            visited[x + 1, y] = 1
                        if x - 1 >= 0 and not visited[x - 1, y] and matrix[x - 1, y] == matrix[x, y]:
                            queue.append((x - 1, y))
                            visited[x - 1, y] = 1
                        if y + 1 < self.size and not visited[x, y + 1] and matrix[x, y + 1] == matrix[x, y]:
                            queue.append((x, y + 1))
                            visited[x, y + 1] = 1
                        if y - 1 >= 0 and not visited[x, y - 1] and matrix[x, y - 1] == matrix[x, y]:
                            queue.append((x, y - 1))
                            visited[x, y - 1] = 1
                     # 筛选出所有在同一行（或列）有三个及以上连续元素的的联通元素组
                    if any(len(set(coord[0] for coord in temp[i:i + 3])) == 1
                           and len(set(coord[1] for coord in temp[i:i + 3])) == 3
                           or len(set(coord[0] for coord in temp[i:i + 3])) == 3
                           and len(set(coord[1] for coord in temp[i:i + 3])) == 1
                           for i in range(len(temp) - 2)):
                        res.append(temp)
                        temp_lines |= set(coord[0] for coord in temp)
        self.lazylines = temp_lines
        return res

    def eliminate_and_score(self):
        """评分后更新

        返回值：
            本轮消除连锁反应得到的分数
        """
        score = 0
        add = 1
        while add:
            list_of_connected_elements = self.scan_for_connected()
            add = sum(map(lambda x: (len(x) - 2) ** 2,
                          list_of_connected_elements))
            # 为了避免多次调用，在这里也顺便把分算了 score:int 总得分
            for sublst in list_of_connected_elements:
                for cordinates in sublst:
                    self.board[cordinates] = 'Q'
            # 把所有的连续同色元素替换成'Q'
            for i in range(self.board.shape[0]):
                row = self.board[i]
                Q_indices = np.where(row == 'Q')[0]
                if len(Q_indices) > 0:
                    row = np.concatenate((row[row != 'Q'], row[Q_indices]))
                    self.board[i] = row
            # 将所有的“Q”移到最上(其实是右）方
            score += add

        self.lazylines = set()
        return score

### 初始化设定
测试，统一设定好种子为40号

In [36]:
np.random.seed(40)
board = Board()

### 1. 贪心算法

In [39]:
def changeable(size):
    """所有可交换的坐标邻居对"""
    pairs = set()
    # 横向的
    for i in range(size - 1):
        for j in range(size):
            pairs.add((i, j, i+1, j))
    for i in range(size):
        for j in range(size - 1):
            pairs.add((i, j, i, j+1))
    return pairs

pairs = changeable(8)

(2, 1, 2, 2) 217


In [ ]:
chosen_score, chosen_pair = 0, None
for pair in pairs:
    board.move(*pair)
    if board.scan_for_connected(): # 贪心剪枝：第一层交换无效的不再copy
        test = board.copy(3) # 最耗时的操作，通常来说最下面的三块棋盘就够了...
        score = test.eliminate_and_score()
        if score > chosen_score:
            chosen_score, chosen_pair = score, pair
    board.move(*pair) # 复原棋盘
print(chosen_pair, chosen_score)

In [ ]:
class Greedy():
    
    def __init__(self) -> None:
        pass